# Preliminary code usage

Highly suggest to use a venv to code this project

To create your venv, install venv if necessary:

```bash
mkdir .venv 
python3 -m venv .venv
source .venv/bin/activate
```

To install the requirements:

```bash
pip install -r requirements.txt
```


Make sure to use .venv python as kernel in jupyter notebook

```bash

## Import libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pyActigraphy

### Magnitude

In [ ]:
data_folder = 'raw_data/'

df = pd.read_csv(data_folder + '1_AHA_RAW.csv')

magnitude_D = np.sqrt(np.square(df['x_D']) + np.square(df['y_D']) + np.square(df['z_D'])) #mano dominante
magnitude_ND = np.sqrt(np.square(df['x_ND']) + np.square(df['y_ND']) + np.square(df['z_ND'])) #mano non dominante

### Operations with time-series' magnitude
- concatenation
- difference (btw domaninat and non dominant)
- asymmetrix index (ai)

In [ ]:
def elaborate_magnitude(operation_type, magnitude_D, magnitude_ND):

    elaborated_magnitude = []

    if operation_type == 'concat':
        elaborated_magnitude = pd.concat([magnitude_D, magnitude_ND], ignore_index=True)
    elif operation_type == 'difference':
        elaborated_magnitude = magnitude_D - magnitude_ND
    elif operation_type == 'ai':
        elaborated_magnitude = (((magnitude_D - magnitude_ND) / (magnitude_D + magnitude_ND)) * 100).fillna(0)
    else: 
        print('operation type non supportata.')
        exit(1)


    return elaborated_magnitude

# Fase di chunking (OPTIONAL)
#Need to select a window size

    for j in range (0, len(magnitude_D), window_size):

        chunk_D = magnitude_D.iloc[j:j + window_size]
        chunk_ND = magnitude_ND.iloc[j:j + window_size]

        if chunk_D.size == window_size and chunk_ND.size == window_size:
            
            for es in estimators:
                es['series'].append(elaborate_magnitude(es['method'], chunk_D, chunk_ND))

            if chunk_D.agg('sum') == 0 and chunk_ND.agg('sum') == 0:
                to_discard.append(int(j/window_size))

### Preprocessing starting from raw data

In [ ]:
# Nella tesi di triennale non fanno un cazzo di pre-processing, prendono direttamente x, y, z dalle time-series



### Correlation analysis

- correlation btw WEEK sessions and Clinical sessions (1h) --> sliding

In [ ]:
#Nella tesi di triennale usano semplicemente corrcoef ma così non fai sliding

